In [1]:
DATASET_DIR = 'dataset'
LABELS = {
    'обеспечение исполнения контракта': 0,
    'обеспечение гарантийных обязательств': 1
}

In [2]:
import json
import pandas as pd
import spacy
from tqdm import tqdm
import re
from spacy import displacy

In [3]:
with open(f'{DATASET_DIR}/train.json', 'rb') as f:
    train = json.load(f)

In [4]:
len(train)

1799

In [ ]:
nlp = spacy.load('ru_core_news_lg')

In [7]:
train[1798]

{'id': 926039184,
 'text': 'Государственная корпорацИя по атомной энергии «росатом» акционерное общество «НИИЭФА им. Д.В. Ефремова» (АО «НИИЭФА») УТВЕРЖДАЮ: Генеральный директор АО «НИИЭФА» ____________________ Е.А. Сакадынец «____» ______________ 2022г. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК (ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ) открытый запрос котировок в электронной форме на право заключения договора на поставку переносного рентгеновского аппарата направленного действия ICM SITE-X D1802 Извещение размещено на официальных сайтах http://www.zakupki.gov.ru, http://zakupki.rosatom.ru и http://roseltorg.ru/ от «07» сентября 2022 года № 220907/0459/___ Санкт-Петербург, 2022г. СОДЕРЖАНИЕ ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАПРОСА КОТИРОВОК 1 1. ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАКУПКИ 3 ЧАСТЬ 1 9 2. ТРЕБОВАНИЯ. ДОКУМЕНТЫ. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ В ЗАКУПКЕ. 9 2.1. ТРЕБОВАНИЯ. ДОКУМЕНТЫ, ПОДТВЕРЖДАЮЩИЕ СООТВЕТСТВИЕ УСТАНОВЛЕННЫМ ТРЕБОВАНИЯМ. 9 2.1.1. Требования к участникам закупки 9 2.1.2. Требования к продукции 12 2.2. 

In [8]:
train[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [9]:
text = train[0]['text']
text

'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 1

In [48]:
ext_text = train[0]['extracted_part']['text'][0]
ext_text_start = train[0]['extracted_part']['answer_start'][0]
ext_text_end = train[0]['extracted_part']['answer_end'][0]

# Предобработка

Идеи:
- Токенизировать текст по словам. Представить каждое слово эмбеддингом. Получим лист эмбеддингов. Трансформировать индексы верных ответов (начало и конец слова) в индексы эмбеддингов. Или трансформировать в эмбеддинги вычленненый текст и сравнивать с эмбеддингами текста, чтобы найти место в тексте.
- Токенизировать текст по предложениям. Представить каждое предложение эмбеддингом.


Для решения данной задачи можно использовать модель семантической сегментации текста на основе машинного обучения. Одним из популярных подходов для решения этой задачи является Sequence Labeling, где каждому слову в тексте присваивается определенный тег в соответствии с его ролью в предложении.

Для решения данной задачи можно использовать модель на основе Conditional Random Fields (CRF). CRF - это статистическая модель, которая используется для прогнозирования последовательностей меток на основе входных данных. Модель учитывает контекст слова и может определить, какой кусок текста в документе соответствует заданному пункту.

В качестве подготовительного этапа, необходимо разметить текст документа, то есть определить метки для каждого слова в тексте, которые соответствуют заданным пунктам. Например, можно использовать BIO-схему разметки (Begin, Inside, Outside), где B соответствует началу куска текста, I - внутренней части куска, а O - всем остальным словам в тексте.

После разметки текста можно обучить модель CRF на этом наборе данных. Обученная модель будет принимать на вход текст документа и название одного из двух пунктов, и возвращать соответствующий кусочек текста из документа. Для удобства использования, можно реализовать данную модель в виде API-сервиса, который будет принимать запросы на получение кусков текста и возвращать соответствующий результат.

# Question Answering

In [2]:
from transformers import pipeline

model_checkpoint = 'timpal0l/mdeberta-v3-base-squad2'
question_answerer = pipeline("question-answering", model=model_checkpoint)

# context = """
# 🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
# between them. It's straightforward to train your models with one before loading them for inference with the other.
# """
# question = "Which deep learning libraries back 🤗 Transformers?"
# question_answerer(question=question, context=context)

ValueError: Could not load model timpal0l/mdeberta-v3-base-squad2 with any of the following classes: (<class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForQuestionAnswering'>, <class 'transformers.models.deberta_v2.modeling_tf_deberta_v2.TFDebertaV2ForQuestionAnswering'>).

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

ImportError: 
AutoModelForQuestionAnswering requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForQuestionAnswering".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


# Sequence Labeling

## Token Bounding

In [61]:
special_tokens = ['<label>', '</label>']
tokens_length = len(special_tokens[0]) + len(special_tokens[1])

In [62]:
def labelize_text_with_tokens(text, start, end):
    start_token, end_token = special_tokens
    return f'{text[:start]}{start_token} {text[start:end]} {end_token}{text[end:]}'

In [92]:
for el in tqdm(train):
    text = el['text']
    ext_text_start = el['extracted_part']['answer_start'][0]
    ext_text_end = el['extracted_part']['answer_end'][0]
    el['labelized_text'] = labelize_text_with_tokens(text, ext_text_start, ext_text_end)
    el['label_id'] = LABELS[el['label']]

In [93]:
train[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [65]:
from transformers import GPT2Tokenizer 


tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B', additional_special_tokens=special_tokens)

def get_tokenized(text, tokenizer):
    return tokenizer.encode(text, add_special_tokens=special_tokens)
                            # , return_tensors='pt', max_length=512, truncation=True, padding='max_length')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [105]:
post_train = []  # TODO: to tensor

special_tokens_ids = tokenizer.encode(special_tokens)

for el in tqdm(train):
    elem = get_tokenized(el['labelized_text'], tokenizer)
    input_elem = elem.copy()
    input_elem.remove(special_tokens_ids[0])
    input_elem.remove(special_tokens_ids[1])
    # TODO: elem to tensor
    post_train.append({
        'input_ids': input_elem,
        'label_id': el['label_id'],
        'start_idx': elem.index(special_tokens_ids[0]),
        'end_idx': elem.index(special_tokens_ids[1]),
    })

100%|██████████| 1799/1799 [00:07<00:00, 236.78it/s]


In [109]:
post_train[78]

{'input_ids': [6014,
  546,
  2357,
  290,
  13957,
  32879,
  42110,
  541,
  37634,
  4855,
  754,
  11275,
  4003,
  646,
  9247,
  22,
  4003,
  3420,
  6695,
  21779,
  5825,
  40428,
  21352,
  2005,
  13530,
  11275,
  4003,
  646,
  9247,
  22,
  4003,
  3420,
  6695,
  640,
  10022,
  1700,
  11407,
  37634,
  824,
  12217,
  8320,
  2632,
  473,
  730,
  1674,
  7719,
  879,
  5172,
  3405,
  18,
  8914,
  18,
  2692,
  2479,
  13,
  23865,
  14296,
  46510,
  473,
  1567,
  989,
  4173,
  16,
  43343,
  13,
  33038,
  451,
  29272,
  640,
  10022,
  1700,
  11410,
  28802,
  8623,
  411,
  33534,
  391,
  1009,
  16,
  3043,
  17937,
  282,
  16866,
  20419,
  6829,
  18,
  38380,
  1357,
  21999,
  10074,
  6929,
  9996,
  2682,
  17788,
  4076,
  16,
  7316,
  1378,
  608,
  1295,
  309,
  6802,
  778,
  2948,
  16,
  20690,
  2733,
  25406,
  282,
  20172,
  16,
  25848,
  547,
  282,
  17597,
  16,
  5063,
  1545,
  28043,
  1518,
  4433,
  4551,
  473,
  26024,
  376,
 

In [103]:
train_0 = [el for el in post_train if el[-1] == 0]
train_1 = [el for el in post_train if el[-1] == 1]
print(len(train_0), len(train_1))


val_percent = 0.1

val_0 = train_0[:int(len(train_0) * val_percent)]
val_1 = train_1[:int(len(train_1) * val_percent)]

train_0 = train_0[int(len(train_0) * val_percent):]
train_1 = train_1[int(len(train_1) * val_percent):]

print(len(train_0), len(val_0), len(train_1), len(val_1))

988 811
890 98 730 81
